# Исследование надежности заемщиков.
Задача: оценка влияния семейного положения и количества детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

Вводные данные:
* `children` — количество детей в семье;
* `days_employed` — общий трудовой стаж в днях;  
* `dob_years` — возраст клиента в годах;
* `education` — уровень образования клиента;
* `education_id` — идентификатор уровня образования;
* `family_status` — семейное положение;
* `family_status_id` — идентификатор семейного положения;
* `gender` — пол клиента;
* `income_type` — тип занятости;
* `debt` — имел ли задолженность по возврату кредитов;
* `total_income` — ежемесячный доход;
* `purpose` — цель получения кредита.

### Шаг 1. Обзор данных

In [1]:
import pandas as pd # импорт библиотеки pandas

In [2]:
df = pd.read_csv('/datasets/data.csv') # Откроем файл с данными и изучим общую информацию.
df.info()
df.to_csv('data.csv', index=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [3]:
df.head(15) # получение первых 15 строк таблицы df

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [4]:
display(df.tail(15)) # получение последних 15 строк таблицы df

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
21510,2,NaN,28,среднее,1,женат / замужем,0,F,сотрудник,0,NaN,приобретение автомобиля
21511,0,-612.569129,29,высшее,0,гражданский брак,1,F,сотрудник,1,140068.472941,покупка жилья для сдачи
21512,0,-165.377752,26,высшее,0,Не женат / не замужем,4,M,компаньон,0,147301.457769,получение дополнительного образования
21513,0,-1166.216789,35,среднее,1,женат / замужем,0,F,сотрудник,0,250986.142309,покупка жилья
21514,0,-280.469996,27,неоконченное высшее,2,Не женат / не замужем,4,M,компаньон,0,355988.407188,строительство недвижимости
21515,1,-467.685130,28,среднее,1,женат / замужем,0,F,сотрудник,1,109486.327999,заняться образованием
21516,0,-914.391429,42,высшее,0,женат / замужем,0,F,компаньон,0,322807.776603,покупка своего жилья
21517,0,-404.679034,42,высшее,0,гражданский брак,1,F,компаньон,0,178059.553491,на покупку своего автомобиля
21518,0,373995.710838,59,СРЕДНЕЕ,1,женат / замужем,0,F,пенсионер,0,153864.650328,сделка с автомобилем
21519,1,-2351.431934,37,ученая степень,4,в разводе,3,M,сотрудник,0,115949.039788,покупка коммерческой недвижимости


**Выводы**

Итак, в таблице двеннадцать столбцов. Тип данных в столбцах различаются — `object`, `float64` и `int64`.

Количество значений в столбцах **`days_employed` и `total_income`** отличается от остальных столбцов. Значит, в данных есть пропущенные значения. В каждой строке таблицы — данные о конкретном клиенте банка. Предварительно можно утверждать, что, данных достаточно для проверки гипотез. 

Но есть проблемы в данных, которые нужно устранить:
* встречаются пропуски в данных;
* в колонке `days_employed` — имеются отрицательные значения;
* в колонке `education` встречается разный регистр при записи данных;
* в столбце `purpose` несколько одних и тех же смысловых значений записаны разными фразами.

Пропуски могли появитьсяв результате как человеческого фактора, так и в результате ошибки системы или процесса выгрузки. 
Чтобы двигаться дальше, нужно устранить проблемы в данных.

### Шаг 2.1 Обработка пропущенных значений

In [5]:
df.isna().sum() # количество пропусков

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

В столбцах **`days_employed`** и **`total_income`** имеем место с пропусками. Возможно это связано с непредоставлением справки с места работы. Также встречаются отрицательные значения, а также слишком большие значения, которые не соответствуют здравому смыслу. Эти данные были либо неправильно извлечены, либо некорректно занесены.  Проверить эти значения и получить корректные данные для нас не представляется возможным. Продолжим работать с тем, что имеем

В столбцах **`days_employed` и `total_income`** пропуски имееют тип NaN. Этот тип относится к вещественным числам, float. Из-за этого `NaN` можно использовать в математических операциях, не вызывая ошибку из-за несовместимых типов.

Пропущенные значения в столбцах лучше заменить меданным значением, так как разброс между min и maх значенимя слмшком велик. Медиана поможет найти именно тот показатель, который ближе всего к истинно среднему. На неё не влияют выбивающиеся из общей массы числа, поэтому она считается одним из самых надёжных и устойчивых показателей.

Проверим какую долю составляют пропущенные значения в каждом из столбцов с пропусками.

In [6]:
number_of_rows = len(df) # количество строк в таблице
missing_values = 2174 # количество пропусков в столбце (одинаковое количество и в столбце days_employed , и в столбце total_income
count_skip = missing_values / number_of_rows 
print('Доля пропусков в столбцах: {:.1%}'.format(count_skip))

Доля пропусков в столбцах: 10.1%


10,1% это значительное количество. Просто удалить эти строки будет не правильно. Нужно будет обработать.

В столбце `days_employed` избавимся от отрицательного знака в даных.

In [7]:
df['days_employed'] = abs(df['days_employed'])

Расчитаем медианное значение для столбцов **`days_employed` и `total_income`**. 

In [8]:
days_employed_median = df['days_employed'].median()
days_employed_median

2194.220566878695

In [9]:
total_income_median = df['total_income'].median()
total_income_median

145017.93753253992

В столбцах `days_employed` и `total_income` заменим пропущенные значения на медианные.

In [10]:
df['days_employed'] = df['days_employed'].fillna(days_employed_median)
#df['total_income'] = df['total_income'].fillna(total_income_median)

In [11]:
for i in df['income_type'].unique():
    df.loc[(df['income_type'] == i) & (df['total_income'].isna()), 'total_income'] = df.loc[(df['income_type'] == i), 'total_income'].median()

Убедимся, что в таблице не осталось пропусков. Для этого ещё раз посчитаем пропущенные значения.

In [12]:
df.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Шаг 2.2 Проверка данных на аномалии и исправления.

Проверим данный методом описательной статистики для поиска всех числовых аномалий

In [13]:
df.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,21525.000000,21525.000000,21525.000000,21525.000000,21525.000000,2.152500e+04
mean,0.538908,60378.032733,43.293380,0.817236,0.972544,0.080883,1.652253e+05
std,1.381587,133257.558514,12.574584,0.548138,1.420324,0.272661,9.804367e+04
min,-1.000000,24.141633,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,1025.608174,33.000000,1.000000,0.000000,0.000000,1.077982e+05
50%,0.000000,2194.220567,42.000000,1.000000,0.000000,0.000000,1.425944e+05
75%,1.000000,4779.587738,53.000000,1.000000,1.000000,0.000000,1.955499e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


In [14]:
df['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

Выявлены аномалии:
* отрицательное количество детей (47 строк);
* сильно выделяющееся значение количества детей - 20 (76 строк).

Можно было бы предположить, что значение **`20`** относится к семьям с приёмными детьми, но такое количество (76) указывает, что это всё же ошибки при занесении данных в таблицу. То есть при нажатии `2` случайно нажимали `0`. 
Отрицателное значение, скорее всего, просто ошибка выгрузки.

Исправим эти значения:
* `-1` заменим на `1`;
* `20` заменим значением, которое по моей гипотезе должно было быть занесено, т.е `2`.

In [15]:
df['children'] = abs(df['children']) #заменим отрицательное значение на положительное

In [16]:
df['children'] = df['children'].replace(20, 2) 

In [17]:
df['children'].value_counts() #проверим исправления

0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64

В столбце `dob_years` имеется значение `0`, хотя кредиты выдаются лицам от 18 лет. Заполнить их можем основываясь на медианные значения, посчитанные для каждого типа занятости. 

In [18]:
dob_years_median = df['dob_years'].median() # расчёт медианы в столбце dob_years
df.loc[df['dob_years'] == 0, 'dob_years'] = dob_years_median # замена нулевого значения медианным
df['dob_years'] = df['dob_years'].astype('int') # перевод из вещественного числа в целочисленное

При изучении таблицы, заметил, что в столбцах присутсвуют значения, записанные разным регистром. Приведем сразу все категории к одному регистру:

In [19]:
df['education'] = df['education'].str.lower()
df['income_type'] = df['income_type'].str.lower()
df['family_status'] = df['family_status'].str.lower()
df['purpose'] = df['purpose'].str.lower()

Проверим, исправили ли регистр выводом количества уникальных значений в столбце

In [20]:
df['gender'].value_counts()

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

В столбце имеются непонятные значения `XNA`. Предположу, что `XNA` ошибка при занесении данных. В столбце преобладающие значения `F`, поэтому заменим `XNA` тоже на `F`.

In [21]:
df['gender'] = df['gender'].replace('XNA', 'F') 

In [22]:
df['gender'].value_counts() #проверим исправления

F    14237
M     7288
Name: gender, dtype: int64

Просмотрим все уникальные значения столбца `purpose`:

In [23]:
df['income_type'].value_counts()

сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
предприниматель        2
безработный            2
в декрете              1
студент                1
Name: income_type, dtype: int64

Тут всё в порядке. Оставим без изменений.

In [24]:
df['purpose'].value_counts()

свадьба                                   797
на проведение свадьбы                     777
сыграть свадьбу                           774
операции с недвижимостью                  676
покупка коммерческой недвижимости         664
покупка жилья для сдачи                   653
операции с жильем                         653
операции с коммерческой недвижимостью     651
жилье                                     647
покупка жилья                             647
покупка жилья для семьи                   641
строительство собственной недвижимости    635
недвижимость                              634
операции со своей недвижимостью           630
строительство жилой недвижимости          626
покупка недвижимости                      624
покупка своего жилья                      620
строительство недвижимости                620
ремонт жилью                              612
покупка жилой недвижимости                607
на покупку своего автомобиля              505
заняться высшим образованием      

Из списка видно, что многие уникальные значения цели кредита записаны по-разному, но имеют одинаковый смысл. Нужна категоризация. Так же есть некорректное написание `жилью`, заменим на `жилья`.

In [25]:
df['purpose'] = df['purpose'].replace('ремонт жилью', 'ремонт жилья') 
df['purpose'].value_counts()

свадьба                                   797
на проведение свадьбы                     777
сыграть свадьбу                           774
операции с недвижимостью                  676
покупка коммерческой недвижимости         664
покупка жилья для сдачи                   653
операции с жильем                         653
операции с коммерческой недвижимостью     651
жилье                                     647
покупка жилья                             647
покупка жилья для семьи                   641
строительство собственной недвижимости    635
недвижимость                              634
операции со своей недвижимостью           630
строительство жилой недвижимости          626
покупка недвижимости                      624
покупка своего жилья                      620
строительство недвижимости                620
ремонт жилья                              612
покупка жилой недвижимости                607
на покупку своего автомобиля              505
заняться высшим образованием      

Проверим столбец **`days_employed`**:

In [26]:
df['days_employed'].value_counts()

2194.220567      2175
329951.594147       1
886.253127          1
2539.534295         1
390574.985524       1
                 ... 
1320.841444         1
1394.302246         1
2325.720832         1
4086.407828         1
1636.419775         1
Name: days_employed, Length: 19351, dtype: int64

In [27]:
df['days_employed'].describe()

count     21525.000000
mean      60378.032733
std      133257.558514
min          24.141633
25%        1025.608174
50%        2194.220567
75%        4779.587738
max      401755.400475
Name: days_employed, dtype: float64

Имеем очень большой разброс в значениях. При этом максимальное значение не соответствует здравому смыслу (около 1100 лет). Плюс в этом столбце изначально данные были с отрицательными значениями. Понимаю, что дальше значения этого столбца мне не понадобятся, пооэтому приму решение просто не тратить время впустую на ненужные вычисления.

In [28]:
df['debt'].value_counts()

0    19784
1     1741
Name: debt, dtype: int64

По этому столбцу  никаких нареканий к занчениям нет. Оставим этот столбец без изменений. Из данных видно, что большинство клиентов не имеет задолженности по кредитам, что говорит о правильной политике кредитной организации по вынесению решения о выдаче кредита.

### Шаг 2.3. Изменение типов данных.

В столбцtе **`total_income`** занесены даные типа `float64`. Но такая точность "до копейки" не играет значения в этих данных. Поэтому преобразуем значения в тип `int`

In [29]:
df['days_employed'] = df['days_employed'].astype('int')
df['total_income'] = df['total_income'].astype('int')

Проверим результат изменения типа данных:

In [30]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21525 non-null  int64 
 1   days_employed     21525 non-null  int64 
 2   dob_years         21525 non-null  int64 
 3   education         21525 non-null  object
 4   education_id      21525 non-null  int64 
 5   family_status     21525 non-null  object
 6   family_status_id  21525 non-null  int64 
 7   gender            21525 non-null  object
 8   income_type       21525 non-null  object
 9   debt              21525 non-null  int64 
 10  total_income      21525 non-null  int64 
 11  purpose           21525 non-null  object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


### Шаг 2.4. Удаление дубликатов.

Посчитаем явные дубликаты в таблице.

In [31]:
print('Дубликатов в таблице:', df.duplicated().sum())

Дубликатов в таблице: 72


Избавимся от дубликатов и перезапишем индексы строк в таблице.

In [32]:
df = df.drop_duplicates().reset_index(drop=True)

Ещё раз посчитаем явные дубликаты в таблице — убедимся, что полностью от них избавились:

In [33]:
print('Дубликатов в таблице:', df.duplicated().sum())

Дубликатов в таблице: 0


### Шаг 2.5. Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма.

Создадим два новых датафрейма, в которых:
* каждому уникальному значению из education соответствует уникальное значение education_id — в первом;
* каждому уникальному значению из family_status соответствует уникальное значение family_status_id — во втором.

In [34]:
education_dict = df[['education', 'education_id']]
education_dict = education_dict.drop_duplicates().sort_values(by = 'education_id').reset_index(drop = True)
education_dict.head()

,education,education_id
0,высшее,0
1,среднее,1
2,неоконченное высшее,2
3,начальное,3
4,ученая степень,4


In [35]:
family_status_dict = df[['family_status', 'family_status_id']]
family_status_dict = family_status_dict.drop_duplicates().sort_values(by = 'family_status_id').reset_index(drop = True)
family_status_dict.head()

,family_status,family_status_id
0,женат / замужем,0
1,гражданский брак,1
2,вдовец / вдова,2
3,в разводе,3
4,не женат / не замужем,4


Удалим из исходного датафрейма столбцы education и family_status, оставив только их идентификаторы: education_id и family_status_id. 

In [36]:
df = df.drop(['education', 'family_status'], axis=1)
df.head()# удаление столбцов и вывод первых 5 строк

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,0,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,1,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,1,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,1,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266,53,1,1,F,пенсионер,0,158616,сыграть свадьбу


### Шаг 2.6. Категоризация дохода.

На основании диапазонов, указанных ниже, создадим столбец total_income_category с категориями:
* 0–30000 — 'E';
* 30001–50000 — 'D';
* 50001–200000 — 'C';
* 200001–1000000 — 'B';
* 1000001 и выше — 'A'.

In [37]:
def total_income_category(total_income):
    if total_income <= 30000:
        return 'E'
    if total_income <= 50000:
        return 'D'
    if total_income <= 200000:
        return 'C'
    if total_income <=1000000:
        return 'B'
    return 'A'    

df['total_income_category'] = df['total_income'].apply(total_income_category)
df.head()

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category
0,1,8437,42,0,0,F,сотрудник,0,253875,покупка жилья,B
1,1,4024,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,C
2,0,5623,33,1,0,M,сотрудник,0,145885,покупка жилья,C
3,3,4124,32,1,0,M,сотрудник,0,267628,дополнительное образование,B
4,0,340266,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,C


Выедем количество уникальных значений столбца `total_income_category`

In [38]:
df['total_income_category'].value_counts()

C    16014
B     5042
D      350
A       25
E       22
Name: total_income_category, dtype: int64

Как видим из данных, больше всего заёмщиков с доходом от 50000 до 200000.

### Шаг 2.7. Категоризация целей кредита.

Создадим функцию, которая на основании данных из столбца `purpose` сформирует новый столбец `purpose_category`, в который войдут следующие категории:
* 'операции с автомобилем';
* 'операции с недвижимостью';
* 'проведение свадьбы';
* 'получение образования'.

In [ ]:
def purpose_group(purpose): # функция категоризации по целям кредита
    if 'образов' in purpose:
        return 'получение образования'
    if 'свадьб' in purpose:
        return 'проведение свадьбы'
    if 'автомобил' in purpose:
        return 'операции с автомобилем'
    if 'жиль' in purpose:
        return 'операции с недвижимостью'
    if 'недвиж' in purpose:
        return 'операции с недвижимостью'
    return 'нет категории'

df['purpose_group'] = df['purpose'].apply(purpose_group) # добавление нового столбца с категоризацией по целям кредита
df.head() # выведение первых 5 строк таблицы

In [ ]:
df['purpose_group'].value_counts()

Из этих данных видно, что "львиную долю" всех кредитов составляют заёмы, связанные с недвижимостью.

### Ответы на вопросы.

Ответим на вопросы:
* Есть ли зависимость между количеством детей и возвратом кредита в срок?
* Есть ли зависимость между семейным положением и возвратом кредита в срок?
* Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
* Как разные цели кредита влияют на его возврат в срок?

##### Вопрос 1: Есть ли зависимость между количеством детей и возвратом кредита в срок?

In [ ]:
def make_proportion(pdSerises): # функция, что посчитает нам отношение в процентах
    return str(round((pdSerises.sum() / pdSerises.count()) * 100, 2)) + '%'

# построим сводную таблицу для ответа на вопрос
data_pivot = df.pivot_table(index=['children'], values=["debt"], aggfunc=['sum', 'count', make_proportion])
# сортируем, чтобы сразу видеть у кого ситуация лучше
data_pivot = data_pivot.sort_values(by=('make_proportion', 'debt'))
data_pivot

**Вывод:**

Небольшая зависимость между количеством детей и просрочкой имеется. Заемщик без детей платит по кредиту в срок чаще, чем заемщик с ребенком/детьми. Но если смотреть по семяьм с детьми, то нельзя сказать, большее количество детей ведёт к вероятности просрочки. Семьи с 1 и 2 детьми имеют больший процент просрочек по сравнению с семьями, у которых 3 и 5 детей. Исходя из этого, строгой зависимости задолженности от количества детей не просматривается.

##### Вопрос 2: Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [ ]:
# вернём ранее удалённый столбуц family_status
df_with_family_status = df.merge(family_status_dict, on='family_status_id', how='left')

# построим сводную таблицу 
data_pivot = df_with_family_status.pivot_table(index=['family_status'], values=["debt"], aggfunc=['sum', 'count', make_proportion])

data_pivot = data_pivot.sort_values(by=('make_proportion', 'debt')) # отсортируем значения
data_pivot

**Вывод:**

Судя по данным, категории "вдовец/вдова" и люди в "разводе" показывают наименьший риск просрочки. Самые "злостные" должники - "неженатые" и состоящие в "гражданском браке". Из этого можно сделать вывод, что чем серьезней отношения - тем меньше шанс просрочки.

##### Вопрос 3: Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [ ]:
# построим сводную таблицу для ответа на вопрос
data_pivot = df.pivot_table(index=['total_income_category'], values=["debt"], aggfunc=['sum', 'count', make_proportion])
data_pivot = data_pivot.sort_values(by=('make_proportion', 'debt')) # отсоритируем
data_pivot

**Вывод:**

Люди с высоким уровнем дохода (более 200000) более ответтсвенны и осуществляют платеж по кредиту в срок. Самые дсициплинированые клиенты с доходом от 30000 до 50000. 
Самые "опасные" клиенты - с самым низким уровнем дохода (оно и понятно) и  сдоходом от 50000 до 200000 (это, так сказать, "СРЕДНИЙ КАЛСС"). Предполагаю, что задолженность среди "СРЕДНЕГО КЛАССА" появляется в следствии завышения людьми показателей реальной заработной платы для получения более крупных кредитов. Из-за этого впоследствии они чаще сталкиваются с проблемами выплаты более крупных сум.

##### Вопрос 4: Как разные цели кредита влияют на его возврат в срок?

In [ ]:
# построим сводную таблицу для ответа на вопрос
data_pivot = df.pivot_table(index=['purpose_group'], values=["debt"], aggfunc=['sum', 'count', make_proportion])
data_pivot = data_pivot.sort_values(by=('make_proportion', 'debt')) # отсортируем для наглядности
data_pivot

**Вывод:**

Самые низкие риски по кредитам, взятым на оперции с недвижимостью. Это может быть связано с процентными ставками - недвижимость обычно приобретается в ипотеку, а ипотека имеет более низкие проценты чем обычный кредит.
Далее идут кредиты на свадьбу, как мы уже выяснили люди состоящие в браке в целом чаще выплачивают кредиты.
Высокое значение просрочек по кредиту на образование может быть связано с тем, что чаще всего люди берут кредит на образование, чтобы в дальнейшем повысить свой уровень дохода, но во время обучения доход может быть низким.
Автомобиль является более рискованной: люди часто не принимают во внимание дополнительные расходы, связанные с эксплуатацией (ТО, непредвиденный ремонт).

## Общий вывод:

Полученные ответы на предыдущие вопросы помогут сформировать образ идеального клиента:
* цель кредита приобретение недвижимости;
* состоящий в браке;
* без детей;
* уровень дохода свыше 20000.

Самыми ненадежными плательщиками являются:
* заемщики, взявшие кредит на покупку автомобиля или на получение образования;
* заемщики, находящиеся в гражданском браке или в статусе "не женат/не замужем".

Выводы выше не означает, что все кто попадает под вышеопределенные параметры обязательно будут иметь задолженности по кредиту. Выделенные выше категории находятся в зоне риска и к таким заемщикам требуется уделить больше внимания, анализируя заемщика комплексно. 